In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
!pip install transformers \
bitsandbytes \
sentencepiece \
transformers[sentencepiece] \
accelerate \
datasets \
trl \
seacrowd\
py7zr # for the samsum data

In [2]:
import torch
import transformers
import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from peft import (
        get_peft_model,
        prepare_model_for_kbit_training,
        LoraConfig
    )
from trl import SFTTrainer

In [3]:
model_name = "microsoft/Phi-3.5-mini-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            load_in_8bit=True,
                                             torch_dtype=torch.float16,
                                            device_map="auto"
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
!pip install seacrowd

In [5]:
data = load_dataset("SEACrowd/indo_general_mt_en_id", trust_remote_code=True)
data_train, data_test, data_val = data["train"], data["test"], data["validation"]
# Randomly select 10k indices
import random
random_indices = random.sample(range(len(data_train)), 10000)

# Select the 10k rows
data_train = data_train.select(random_indices)
print(data_train, data_test, data_val)

# example
data_train[0]

Dataset({
    features: ['id', 'src', 'tgt'],
    num_rows: 10000
}) Dataset({
    features: ['id', 'src', 'tgt'],
    num_rows: 2000
}) Dataset({
    features: ['id', 'src', 'tgt'],
    num_rows: 2000
})


{'id': '1336521',
 'src': 'These bombs were viscous at the moment of ejection and by rotation in the air acquired their shape.',
 'tgt': 'Bom beruwujud kental pada saat keluar dan akibat rotasi di udara diperoleh-lah bentuk yang ada sekarang.'}

In [6]:
def generate_prompt(dialogue, output=None, eos_token="</s>"):
  instruction = """
Translate the following sentences from English to Indonesian:

English: I am going to the market.
Indonesian: Saya akan pergi ke pasar.

English: """
  input = f"{dialogue}\n"
  output = f"""Indonesian:
#answer: {output + ' ' + eos_token if output else ''} """
  prompt = (" ").join([instruction, input, output])
  return prompt

print(generate_prompt(data_train[0]["src"], data_train[0]["tgt"]))
# Summarize the following:
# Amanda: I baked  cookies. Do you want some?
# Jerry: Sure!
# Amanda: I'll bring you tomorrow :-)
#  Summary: Amanda baked cookies and will bring Jerry some tomorrow.


Translate the following sentences from English to Indonesian:

English: I am going to the market.
Indonesian: Saya akan pergi ke pasar.

English:  These bombs were viscous at the moment of ejection and by rotation in the air acquired their shape.
 Indonesian:
#answer: Bom beruwujud kental pada saat keluar dan akibat rotasi di udara diperoleh-lah bentuk yang ada sekarang. </s> 


In [7]:
input_prompt = generate_prompt(data_train[50]["src"])
input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
with torch.cuda.amp.autocast():
  generation_output = model.generate(
      input_ids=input_tokens,
      max_new_tokens=1000,
      do_sample=True,
      top_k=10,
      top_p=0.9,
      temperature=0.3,
      repetition_penalty=1.15,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
    )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)
# Summarize the following:
#  Pitt: Hey Teddy! Have you received my message?
# Teddy: No. An email?
# Pitt: No. On the FB messenger.
# Teddy: Let me check.
# Teddy: Yeah. Ta!
# Summary: 1) What is the purpose of this conversation? 2) Who initiates it and who responds to it? 3) How do they communicate with each other? 4) What does the speaker think about his friend’s response?

<ipython-input-7-542f0c84d9ad>:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Translate the following sentences from English to Indonesian:

English: I am going to the market.
Indonesian: Saya akan pergi ke pasar.

English:  "Previously Bosaso was known as Bandar Qasim, a name derived from an Arab trader of the same name who is said to have settled in the area during the 14th century."
 Indonesian:
#answer:   Bahasa Inggris :"Pada awalnya bosaso dikenali sebagai Bander Kasi, nama yang berasal dari pelaku alir-alisi arab tersebut bernama saksi telah menetap dalam daerah itu pada abad XIV".
Bahasa Indonesia : Pada awalnya Bosaso dicatatan dengan namanya Bandara Kasih saja, nama berguncangan olehpelikumpultrukulerarabterusya dan klaimsannainyalitukadalampaupunabdadiyuruhmenletempatinda atasperdagungareaitumadupperdibentilabel XIX .


In [8]:
# lora_config = LoraConfig(
#         r=8,
#         lora_alpha=8,
#         lora_dropout=0.1,
#         target_modules=["q_proj","k_proj","v_proj","o_proj"],
#         bias="none",
#         task_type="CAUSAL_LM",
#     )

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["qkv_proj", "o_proj"],  # Adjust based on your model architecture
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
    # bnb_8bit=True  # Enable bitsandbytes 8-bit quantization
)

In [9]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3SdpaAttention(
          (o_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear8bitLt(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear8bitLt(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (no

In [10]:
# this should be set for finutning and batched inference
tokenizer.add_special_tokens({"pad_token": "<PAD>"})
model.resize_token_embeddings(len(tokenizer))

Embedding(32012, 3072, padding_idx=32000)

In [11]:
pip install -U peft

In [12]:
# Loading in 8 bit ..."
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [13]:
output_dir = "cps"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
per_device_eval_batch_size = 4
eval_accumulation_steps = 4
optim = "adamw_hf"
save_steps = 10
logging_steps = 10
learning_rate = 5e-4
max_grad_norm = 0.3
max_steps = 50
warmup_ratio = 0.03
evaluation_strategy="epoch"
lr_scheduler_type = "constant"

training_args = transformers.TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            optim=optim,
            evaluation_strategy=evaluation_strategy,
            save_steps=save_steps,
            learning_rate=learning_rate,
            logging_steps=logging_steps,
            max_grad_norm=max_grad_norm,
            num_train_epochs=40,
            # max_steps=max_steps,
            fp16=True,
            warmup_ratio=warmup_ratio,
            group_by_length=True,
            lr_scheduler_type=lr_scheduler_type,
            ddp_find_unused_parameters=False,
            eval_accumulation_steps=eval_accumulation_steps,
            per_device_eval_batch_size=per_device_eval_batch_size,
        )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
def formatting_func(prompt):
  output = []

  for d, s in zip(prompt["src"], prompt["tgt"]):
    op = generate_prompt(d, s)
    output.append(op)

  return output


trainer = SFTTrainer(
    model=model,
    train_dataset=data_train,
    eval_dataset=data_val,
    peft_config=lora_config,
    formatting_func=formatting_func,
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args
)

# We will also pre-process the model by upcasting the layer norms in float 32 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core fo

<IPython.core.display.Javascript object>

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,1.092100,1.079701


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/peft/

KeyboardInterrupt: 

In [ ]:
trainer.save_model(f"{output_dir}/final")

In [ ]:
from peft import PeftModel

peft_model_id = "cp/checkpoint-40"
peft_model = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16, offload_folder="lora_results/lora_7/temp")

In [ ]:
input_prompt = generate_prompt(data_test[3]["src"])
input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
with torch.cuda.amp.autocast():
    generation_output = peft_model.generate(
        input_ids=input_tokens,
        max_new_tokens=100,
        do_sample=True,
        top_k=10,
        top_p=0.9,
        temperature=0.3,
        repetition_penalty=1.15,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
      )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# Move 'my_folder' to 'new_folder'
!mv "./cps/checkpoint-620" "/content/drive/MyDrive/cps/Phi3.5-40-epoch"

In [ ]:
trainer.save_model(f"/content/drive/MyDrive/{output_dir}/Phi3.5-40-epoch")